## _*Quantum Emoticon*_ 

For an explanation of what I did and why, see [here](https://medium.com/@decodoku/making-a-quantum-computer-smile-cee86a6fc1de).

For more information about how to use [Qiskit](https://qiskit.org/) and [IBM Quantum Services](https://quantum-computing.ibm.com/), see the [Qiskit textbook](https://qiskit.org/textbook/preface.html).

***
### Contributors
* James R. Wootton, University of Basel (the original 2017 version)
* James R. Wootton, IBM Quantum (the 2021 update).

***

This program aims to do some of the same jobs as "Hello World" does for classical programming.

Specifically:
* It is simple;
* It performs and understandable and relatable task;
* It demonstrates simple principles of the programming language;
* It shows how to produce and look at an output.

Software for quantum computers is made up of what we call *quantum circuits*. To make these, we'll use the `QuantumCircuit` object from Qiskit.

In [ ]:
from qiskit import QuantumCircuit

We'll also need to choose where to run our quantum circuit. 

Ideally you'll use real quantum hardware with 16 qubits. IBM Quantum's `ibmq_guadalupe` is up to the task. You can get it ready to use with the following lines of code.

```
from qiskit import IBMQ
IBMQ.load_account()

for provider in IBMQ.providers():
    for device in provider.backends():
        if device.name()=='ibmq_guadalupe':
            backend = device
```

You can probably do it much more efficiently too. But that's how I do it!

If the quantum circuit is small and simple enough, another option is to get a normal computer to simulate what a perfect quantum computer would do. To set this up, use the following lines of code.

```
from qiskit import Aer

backend = Aer.get_backend('qasm_simulator')
```

You can also get a normal computer to pretend to be a real quantum device, with imperfections and all. That's what we'll do, faking the `ibmq_mumbai` device, so we'll run the lines of code below.

In [ ]:
from qiskit.test.mock import FakeMumbai

backend = FakeMumbai()

Writing "Hello World" onto qubits is not possible with current technology. ASCII encoding of "Hello World" requires nearly 100 bits, and storing each classical bit value requires a qubit. There is therefore simple not enough qubits on any current device.

However, two ASCII characters require only 16 (qu)bits. Writing an emoticon like ;) can therefore be done using *ibmqx5*.

The string corresponding to ;) is
    
    ;)  =  '0011101100101001'

Since all qubits are initialized in state 0, writing this bit string using qubits requires us to apply a NOT gate to every qubit that corresponds to a bit in state 1. We can do this using the gate X. If we number the bits from 0 to 15, from right to left, this an X on qubits 0, 3, 5, 8, 9, 11, 12, 13. 

Of course, just writing a bit string in some qubits isn't very quantum. Instead we can prepare a superposition of two different emoticons. We choose ;) and 8), which correspond to the bit strings

    8)  =  '0011100000101001'
    ;)  =  '0011101100101001'

Note that these strings differ only on bits 8 and 9. It is therefore only these on which the superposition must be prepared. The superposition will be of the '00' of 8), and the '11' of ;) , and so will be a standard [Bell state](https://github.com/QISKit/qiskit-tutorial/blob/master/2_quantum_information/superposition_and_entanglement.ipynb).

We'll now implement this and run it for 1024 shots.

In [ ]:
# set up a quantum circuit with 16 qubits
qc = QuantumCircuit(16)

# rightmost eight (qu)bits have ')' = `00101001`
# so flip the three that need to be `1`
qc.x([0,3,5])


# second eight (qu)bits have superposition of
# '8' = 00111000
# ';' = 00111011
# so flip the three that are definitely `1`s
qc.x([11,12,13])
# the rightmost two bits differ, so we create superposition on 9
qc.h(9)
qc.cx(9,8) # spread it to 8 with a cnot

# measure
qc.measure_all()

# run and get results
job = backend.run([qc])
stats = job.result().get_counts()

The results in *stats* tell us how many times each bit string was found as a result. To print the result in an understandable way, we can convert the bit strings into ASCII characters. We can also calculate the fraction of shots for which each result occurred. This processed data makes up *emoticonDict*.

In [ ]:
characterDict = {}
for bitString in stats: # loop over all results
    char1 = chr(int( bitString[0:8] ,2)) # get string of leftmost 8 bits an convert to an ASCII character
    char2 = chr(int( bitString[8:16] ,2)) # same for string of leftmost 8 bits
    characterDict[ char1 + char2 ] = stats[bitString] / 1024

In [ ]:
characterDict = {'h)': 0.0009765625, '8é': 0.0009765625, ':+': 0.0009765625, '8i': 0.0009765625, ';©': 0.001953125, '2)': 0.0009765625, ':(': 0.0009765625, ';$': 0.0009765625, '©-': 0.0009765625, '8©': 0.001953125, '(a': 0.0009765625, 'é)': 0.0009765625, '8¡': 0.0009765625, 'ø)': 0.0029296875, '+\t': 0.0009765625, 'û)': 0.0029296875, ';!': 0.015625, '\x1b)': 0.005859375, '\x9b)': 0.0009765625, '\x18)': 0.0068359375, '+)': 0.013671875, '<(': 0.0009765625, '<)': 0.00390625, '=)': 0.001953125, '¹)': 0.0009765625, '8)': 0.2939453125, '9)': 0.0322265625, '<!': 0.0009765625, '¸)': 0.0224609375, '{)': 0.0380859375, 'x(': 0.0009765625, '¨)': 0.0009765625, '?(': 0.001953125, '[)': 0.001953125, '7)': 0.001953125, '?)': 0.0087890625, ';\x89': 0.0009765625, '1)': 0.0009765625, ';)': 0.3056640625, '3\t': 0.0009765625, '8!': 0.0087890625, '8\t': 0.0068359375, '0)': 0.0068359375, '{!': 0.001953125, '«)': 0.001953125, ';i': 0.00390625, ':!': 0.0029296875, 'k)': 0.001953125, '»)': 0.0224609375, '8(': 0.0029296875, '|)': 0.0009765625, '\x9b!': 0.0009765625, ';9': 0.001953125, ';(': 0.0068359375, 'q)': 0.0009765625, 'x!': 0.0009765625, '()': 0.009765625, ':)': 0.025390625, '?i': 0.0009765625, '»(': 0.0009765625, '¿©': 0.0009765625, '(+': 0.0009765625, '«!': 0.0009765625, '8+': 0.005859375, '»!': 0.0009765625, ';\t': 0.0068359375, '9!': 0.001953125, '9+': 0.0009765625, '((': 0.0009765625, 's)': 0.0009765625, 'x)': 0.033203125, '¿)': 0.0029296875, '3)': 0.0029296875, '>(': 0.0009765625, '+!': 0.0009765625, 'X)': 0.0029296875, '»+': 0.0009765625, '{i': 0.0009765625, 'y)': 0.00390625, '¼)': 0.0009765625, 'z)': 0.0029296875, '»©': 0.0009765625, 'º9': 0.0009765625, '89': 0.0009765625, ';+': 0.0029296875, '\x19)': 0.001953125, '\x18(': 0.0009765625, '0!': 0.0009765625, 'ú+': 0.0009765625, '8-': 0.0009765625, 'x\t': 0.0009765625, '0(': 0.0009765625, '?+': 0.0009765625, '\x18!': 0.001953125, '?©': 0.0009765625, '9\t': 0.001953125, ';-': 0.0068359375, '[!': 0.0009765625, 'º)': 0.0009765625, '+©': 0.0009765625}


Finally it is time to make an actual emoticon. We'll make an image in which the all the characters included in the result are printed on top of each other. The alpha channel is set to the fraction of shots for which that result occurred. Ideally this should be equally split between 8) and ;), but noise will mean some other stuff will turn up too.

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='monospace')
for char in characterDict.keys():
    # plot all characters on top of each other with alpha given by how often it turned up in the output
    plt.annotate( char, (0.5,0.5), va="center", ha="center", color = (0,0,0,characterDict[char]), size = 300)
plt.axis('off')
plt.show()

And there you have it. We took one of the world's most advanced devices and made a smiley with it. A quantum smiley!

To see exactly what the main contributions to this are, we can look at which results occurred in more than 5% of shots.

In [ ]:
for char in characterDict.keys():
    if (characterDict[char]>0.05):
        print(characterDict[char],char)